In [1]:
import base64
import requests
from urllib.parse import urlencode

In [2]:
client_id = ''
client_secret = ''

In [3]:
class SpotifySearchResultItem():
    
    def __init__(self, name, weblink, image={}):
        self.name = name
        self.weblink = weblink
        self.image = image 
    
    def __str__(self):
        return str(self.__dict__)

class Track(SpotifySearchResultItem):

    def __init__(self, name, weblink, image, artist, album, release_date):
        super().__init__(name, weblink, image)
        self.artist = artist
        self.album = album
        self.release_date = release_date

class Album(SpotifySearchResultItem):

    def __init__(self, name, weblink, image, artist, release_date):
        super().__init__(name, weblink, image)
        self.artist = artist
        self.release_date = release_date

class Artist(SpotifySearchResultItem):

    def __init__(self, name, weblink, image, genres, followers):
        super().__init__(name, weblink, image)
        self.genres = genres
        self.followers = followers


In [4]:
a = Album('Analogue', 'https://open.spotify.com/album/1SgtXskC74No0x9a9iDlfw', {'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273d0cbdb71222a11eabce96a70', 'width': 640}, 
         'ODIE', '2019')
print(a.image)

b = Track('Faith', 'google.com', {'h':800}, 'ODIE', 'Analogue', '2019')
print(b.name)

c=Artist('ODIE', 'google.com', {'h':800}, ['rap', 'hip-hop'], 200405 )
print(c.followers)

print(c)

{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273d0cbdb71222a11eabce96a70', 'width': 640}
Faith
200405
{'name': 'ODIE', 'weblink': 'google.com', 'image': {'h': 800}, 'genres': ['rap', 'hip-hop'], 'followers': 200405}


In [5]:
class SpotifyAPI():
    client_id = None
    client_secret = None
    access_token = None
    
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        client_id = self.client_id
        client_secret = self.client_secret
        client_creds = f'{client_id}:{client_secret}'
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_header(self):
        client_creds_b64 = self.get_client_credentials()
        return {'Authorization': f'Basic {client_creds_b64}'}
    
    def get_token_data(self):
        return {'grant_type':'client_credentials'}
    
    def perform_auth(self):
        token_url = 'https://accounts.spotify.com/api/token'
        token_data = self.get_token_data()
        token_header = self.get_token_header()
        r = requests.post(token_url, data=token_data, headers=token_header)
        if r.status_code in range(200, 299):
            token_response_data = r.json()
            self.access_token = token_response_data['access_token']
            print("success")
    
    def get_resource_header(self):
        headers = {
            "Authorization": f"Bearer {self.access_token}"
        }
        return headers
    
    def get_artist_data(self, artist):
        try:
            name = artist['name']
            weblink = artist['external_urls']['spotify']
            image = artist['images'][0]
            genres = artist['genres']
            followers = artist['followers']['total']
            artist_data = Artist(name, weblink, image, genres, followers)
        except:
            return False
        return artist_data 
    
    def get_artists(self, data):
        artists_search_result = []
        for artist in data['artists']['items']:
            artist_data = self.get_artist_data(artist)
            if artist_data == False:
                continue
            else:
                artists_search_result.append(artist_data)
        return artists_search_result
    
    def get_album_data(self, album):
        try:
            name = album['name']
            artist = album['artists'][0]['name']
            weblink = album['external_urls']['spotify']
            image = album['images'][0]
            release_date = album['release_date']
            album_data = Album(name, weblink, image, artist, release_date)
        except:
            return False
        return album_data
    
    def get_albums(self, data):
        albums_search_result = []
        for album in data['albums']['items']:
            album_data = self.get_album_data(album)
            if album_data == False:
                continue
            else: 
                albums_search_result.append(album_data)
        return albums_search_result
        
    def get_track_data(self, track):
        try:
            name = track['name']
            artist = track['artists'][0]['name']
            weblink = track['external_urls']['spotify']
            image = track['album']['images'][0]
            album = track['album']['name']
            release_date = track['album']['release_date']
            track_data = Track(name, weblink, image, artist, album, release_date)
        except:
            return False
        return track_data

    def get_tracks(self, data):
        tracks_search_result = []
        for track in data['tracks']['items']:
            track_data = self.get_track_data(track)
            if track_data == False:
                continue
            else:
                tracks_search_result.append(track_data)
        return tracks_search_result
    
    def base_search(self, query, search_type):
        headers = self.get_resource_header()
        query = " ".join([f"{k}:{v}" for k,v in query.items()])
        endpoint = 'https://api.spotify.com/v1/search'
        query_params = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f'{endpoint}?{query_params}'
        r = requests.get(lookup_url, headers=headers)
        return r.json()
    
    def search(self, query, search_type):
        search_type = search_type.lower()
        data = self.base_search(query, search_type)
        if search_type == 'artist':
            search_result = self.get_artists(data)
        elif search_type == 'album':
            search_result = self.get_albums(data)
        else:
            search_result = self.get_tracks(data)
        # print(search_result)
        return search_result
        
        

In [6]:
spotify = SpotifyAPI(client_id, client_secret)

In [7]:
spotify.perform_auth()

success


In [11]:
artists = spotify.search({'artist': 'ODIE'}, 'artist')
for artist in artists:
    print(artist)
    print()

{'name': 'ODIE', 'weblink': 'https://open.spotify.com/artist/5aMIbwZQvP2MHPMVC5zCGj', 'image': {'height': 640, 'url': 'https://i.scdn.co/image/a21c20c30b55b31aff3f6aa84a6e7156f25363e7', 'width': 640}, 'genres': ['alternative r&b', 'indie r&b', 'trap soul'], 'followers': 97142}

{'name': 'Callie Odie', 'weblink': 'https://open.spotify.com/artist/1XZRepVs1NXhC3ITI79j7m', 'image': {'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273e41891392874de04512aefa5', 'width': 640}, 'genres': [], 'followers': 0}

{'name': 'Odie Ray', 'weblink': 'https://open.spotify.com/artist/5u7TFvIcykj1uOylQZmbhA', 'image': {'height': 640, 'url': 'https://i.scdn.co/image/ce829db43b794e3233582c05cf62581b7c1b2cea', 'width': 640}, 'genres': [], 'followers': 42}

{'name': 'Odiemagee', 'weblink': 'https://open.spotify.com/artist/47EdZXpOvhXQ6S8ZasUAtr', 'image': {'height': 640, 'url': 'https://i.scdn.co/image/b7d67dbb45b4b4158763762e7e473ecc22f19760', 'width': 640}, 'genres': [], 'followers': 24}

{'name'

In [12]:
albums =spotify.search({'artist': 'ODIE', 'album':'Analogue'}, 'album')
for album in albums:
    print(album)
    print()

{'name': 'Analogue', 'weblink': 'https://open.spotify.com/album/1SgtXskC74No0x9a9iDlfw', 'image': {'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273d0cbdb71222a11eabce96a70', 'width': 640}, 'artist': 'ODIE', 'release_date': '2018-04-06'}



In [13]:
tracks = spotify.search({'artist': 'ODIE', 'album':'Analogue', 'track': 'Faith'}, 'track')
for track in tracks:
    print(track)
    print()

{'name': 'Faith', 'weblink': 'https://open.spotify.com/track/3arxPA32v1n0iWnOAHEBM3', 'image': {'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273d0cbdb71222a11eabce96a70', 'width': 640}, 'artist': 'ODIE', 'album': 'Analogue', 'release_date': '2018-04-06'}

